In [4]:
import os
import warnings
warnings.filterwarnings(action='ignore')

if os.getcwd() == '/content':
    data_path = '/content/drive/MyDrive/Colab Notebooks/datasets'
else:
    data_path = r'G:\내 드라이브\Colab Notebooks\datasets'

In [9]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [10]:
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [11]:
# hyper-params
input_size = 784 # 28 x 28
hidden_size = 100
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [8]:
# MNIST
train_dataset = datasets.MNIST(root=data_path, train=True, download=True,
                               transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root=data_path, train=True, download=True,
                              transform=transforms.ToTensor())



SyntaxError: invalid syntax (2889738004.py, line 7)